In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from oasis.functions import deconvolve
from scipy.stats import pearsonr

from utils.context_data_csv import ContextDataCSV
from utils.matrix_optimizer import MatrixOptimizer

import utils.sulfur.constant as const

/app/OASIS/oasis/functions.py:13: UserWarning: Could not find cvxpy. Don't worry, you can still use OASIS, just not the slower interior point methods we compared to in the papers.
  warn("Could not find cvxpy. Don't worry, you can still use OASIS, " +


In [2]:
index = const.ANIMAL_NAMES
columns_df = pd.DataFrame()
contexts = ['A1pre', 'A1post', 'A2_start', 'A2_end', 'A3_start', 'A3_end', 'A4pre', 'A4post', 'A5', 'A1_1', 'A1_2', 'A1_3', 'A4_1', 'A4_2', 'A4_3']
features = ['clustering coefficient', 'shortest path length', 'co-firing strength', 'average weight', 'average edges', 'assortativity']
context_columns = []
for context_name in contexts:
    context_columns += [context_name] * len(features)
columns_df.loc[:, 'context'] = context_columns
columns_df.loc[:, 'feature'] = features * len(contexts)
columns = pd.MultiIndex.from_frame(columns_df)

df = pd.DataFrame(0.0, index=index, columns=columns)

In [3]:
def fetch_preserved_shock_values(shock_cell_name, preserved_shocks):
    if shock_cell_name in preserved_shocks:
        return preserved_shocks[shock_cell_name]
    
    return None

def estimate_spike(matrix, cell_name):
    spike_threshold = 0.05
    
    values = matrix.loc[:, cell_name]
    
    if values.isnull().all():
        return np.zeros(len(values))
    
    _c, spikes, _b, _g, _lam = deconvolve(values.to_numpy(), g=(None, None), penalty=1)
    spikes[spikes <= spike_threshold] = 0.0
    
    return spikes

def append_preserved_shock_values(shock_cell_name, preserved_shocks, spikes):
    if shock_cell_name in preserved_shocks:
        return
    
    preserved_shocks[shock_cell_name] = spikes
    
def is_all_zeros_constant_array(array):
    return np.all(array == 0.0)

def calc_cofiring_graph(matrix, shock_cells, preserved_spikes, start_frame, end_frame):
    spike_threshold = 0.05
    pearsonr_threshold = 0.1
    graph = nx.Graph()
    
    for i in range(len(shock_cells)):
        base_shock_cell = shock_cells[i]
        
        base_spikes = fetch_preserved_shock_values(base_shock_cell, preserved_spikes)
        if base_spikes is None:
            base_spikes = estimate_spike(matrix, base_shock_cell)
            
        append_preserved_shock_values(base_shock_cell, preserved_spikes, base_spikes)
        
        base_spikes = base_spikes[start_frame:end_frame]
        if is_all_zeros_constant_array(base_spikes):
            continue
        
        for j in range(i + 1, len(shock_cells)):
            compared_shock_cell = shock_cells[j]
            
            compared_spikes = fetch_preserved_shock_values(compared_shock_cell, preserved_spikes)
            if compared_spikes is None:
                compared_spikes = estimate_spike(matrix, compared_shock_cell)
                
            append_preserved_shock_values(compared_shock_cell, preserved_spikes, compared_spikes)
            
            compared_spikes = compared_spikes[start_frame:end_frame]
            
            if is_all_zeros_constant_array(compared_spikes):
                continue

            r, _ = pearsonr(base_spikes, compared_spikes)
            if r >= pearsonr_threshold:
                if base_shock_cell not in graph.nodes:
                    graph.add_node(base_shock_cell)
                graph.add_edge(base_shock_cell, compared_shock_cell, weight=r, length=1/r)
                
    return graph
    
def average_path_length(graph, weight='length'):
    def path_lengths(graph, node, weight='length'):
        lengths = []
        for target in graph:
            if not nx.has_path(graph, node, target):
                continue
                
            lengths.append(nx.dijkstra_path_length(graph, node, target, weight=weight))
        
        return lengths
        
    all_lengths = np.array([])
    for node in graph:
        for length in path_lengths(graph, node, weight=weight):
            all_lengths = np.append(all_lengths, length)
    return np.mean(all_lengths)

def calc_features(animal_name, context_name, graph, df):
    df.loc[animal_name, (context_name, 'clustering coefficient')] += nx.average_clustering(graph, weight='weight')
    df.loc[animal_name, (context_name, 'shortest path length')] += average_path_length(graph, weight='length')
    
    adjacency_matrix = pd.DataFrame(nx.adjacency_matrix(graph).todense())
    n = len(graph.nodes)
    df.loc[animal_name, (context_name, 'co-firing strength')] += adjacency_matrix.sum().mean()
    df.loc[animal_name, (context_name, 'assortativity')] += nx.degree_pearson_correlation_coefficient(graph, weight='weight')
    df.loc[animal_name, (context_name, 'average weight')] += sum(weight for _, _, weight in graph.edges.data("weight")) / (n * (n - 1))
    df.loc[animal_name, (context_name, 'average edges')] += len(graph.edges.data("weight")) / n
    
def average_features(animal_name, context_name, df, size):
    df.loc[animal_name, (context_name, 'clustering coefficient')] /= size
    df.loc[animal_name, (context_name, 'shortest path length')] /= size
    df.loc[animal_name, (context_name, 'co-firing strength')] /= size
    df.loc[animal_name, (context_name, 'assortativity')] /= size
    df.loc[animal_name, (context_name, 'average weight')] /= size
    df.loc[animal_name, (context_name, 'average edges')] /= size
    
def all_context_graph(matrix, shock_cells, preserved_spikes, animal_name, context_name, df, start_second=0):
    iter_size = 100
    frame_range = 20
    
    for i in range(start_second, iter_size + start_second):
        index = i * frame_range
        graph = calc_cofiring_graph(matrix, shock_cells, preserved_spikes, index, index + frame_range)
        calc_features(animal_name, context_name, graph, df)
    average_features(animal_name, context_name, df, iter_size)

In [4]:
for animal_name in const.ANIMAL_NAMES:
    print('animal_name: %s' % animal_name)
    
    a1pre_csv = ContextDataCSV(animal_name, 'A1preES')
    a1pre_matrix = MatrixOptimizer(a1pre_csv.data_frame).divide_sd()
    a1pre_shock_matrix = a1pre_matrix
    
    shock_cells = a1pre_csv.cell_names
    
    a1pre_preserved_spikes = {}
    all_context_graph(a1pre_shock_matrix, shock_cells, a1pre_preserved_spikes, animal_name, 'A1pre', df)
    
    a1_csv = ContextDataCSV(animal_name, 'A1postES')
    a1_matrix = MatrixOptimizer(a1_csv.data_frame).divide_sd()
    a1_shock_matrix = a1_matrix
    
    a1_preserved_spikes = {}
    all_context_graph(a1_shock_matrix, shock_cells, a1_preserved_spikes, animal_name, 'A1post', df)
    
    a1_1_graph = calc_cofiring_graph(a1_shock_matrix, shock_cells, a1_preserved_spikes, 80, 100)
    a1_2_graph = calc_cofiring_graph(a1_shock_matrix, shock_cells, a1_preserved_spikes, 980, 1000)
    a1_3_graph = calc_cofiring_graph(a1_shock_matrix, shock_cells, a1_preserved_spikes, 1880, 1900)
    
    calc_features(animal_name, 'A1_1', a1_1_graph, df)
    calc_features(animal_name, 'A1_2', a1_2_graph, df)
    calc_features(animal_name, 'A1_3', a1_3_graph, df)
    
    a2_1_csv = ContextDataCSV(animal_name, 'A2_1')
    a2_1_matrix = MatrixOptimizer(a2_1_csv.data_frame).divide_sd()
    a2_1_shock_matrix = a2_1_matrix
    
    a2_1_preserved_spikes = {}
    all_context_graph(a2_1_shock_matrix, shock_cells, a2_1_preserved_spikes, animal_name, 'A2_start', df)
    
    a2_2_csv = ContextDataCSV(animal_name, 'A2_2')
    a2_2_matrix = MatrixOptimizer(a2_2_csv.data_frame).divide_sd()
    a2_2_shock_matrix = a2_2_matrix
    
    a2_2_preserved_spikes = {}
    all_context_graph(a2_2_shock_matrix, shock_cells, a2_2_preserved_spikes, animal_name, 'A2_end', df)
    
    a3_1_csv = ContextDataCSV(animal_name, 'A3_1')
    a3_1_matrix = MatrixOptimizer(a3_1_csv.data_frame).divide_sd()
    a3_1_shock_matrix = a3_1_matrix
    
    a3_1_preserved_spikes = {}
    all_context_graph(a3_1_shock_matrix, shock_cells, a3_1_preserved_spikes, animal_name, 'A3_start', df)
    
    a3_2_csv = ContextDataCSV(animal_name, 'A3_2')
    a3_2_matrix = MatrixOptimizer(a3_2_csv.data_frame).divide_sd()
    a3_2_shock_matrix = a3_2_matrix
    
    a3_2_preserved_spikes = {}
    all_context_graph(a3_2_shock_matrix, shock_cells, a3_2_preserved_spikes, animal_name, 'A3_end', df)
    
    a4pre_csv = ContextDataCSV(animal_name, 'A4preES')
    a4pre_matrix = MatrixOptimizer(a4pre_csv.data_frame).divide_sd()
    a4pre_shock_matrix = a4pre_matrix
    
    a4pre_preserved_spikes = {}
    all_context_graph(a4pre_shock_matrix, shock_cells, a4pre_preserved_spikes, animal_name, 'A4pre', df)
    
    a4_csv = ContextDataCSV(animal_name, 'A4postES')
    a4_matrix = MatrixOptimizer(a4_csv.data_frame).divide_sd()
    a4_shock_matrix = a4_matrix
    
    a4_preserved_spikes = {}
    all_context_graph(a4_shock_matrix, shock_cells, a4_preserved_spikes, animal_name, 'A4post', df)
    
    a4_1_graph = calc_cofiring_graph(a4_shock_matrix, shock_cells, a4_preserved_spikes, 80, 100)
    a4_2_graph = calc_cofiring_graph(a4_shock_matrix, shock_cells, a4_preserved_spikes, 980, 1000)
    a4_3_graph = calc_cofiring_graph(a4_shock_matrix, shock_cells, a4_preserved_spikes, 1880, 1900)
    
    calc_features(animal_name, 'A4_1', a4_1_graph, df)
    calc_features(animal_name, 'A4_2', a4_2_graph, df)
    calc_features(animal_name, 'A4_3', a4_3_graph, df)
    
    a5_csv = ContextDataCSV(animal_name, 'A5')
    a5_matrix = MatrixOptimizer(a5_csv.data_frame).divide_sd()
    a5_shock_matrix = a5_matrix
    
    a5_preserved_spikes = {}
    all_context_graph(a5_shock_matrix, shock_cells, a5_preserved_spikes, animal_name, 'A5', df)
    
df

animal_name: ID181106CreA


KeyboardInterrupt: 

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(8, 6), tight_layout=True, dpi=200)

x = ['A1pre', 'A1post', 'A2_start', 'A2_end', 'A3_start', 'A3_end', 'A4pre', 'A4post', 'A5']
a = [0, 1, 2, 3, 4, 6, 7, 8, 9]
summary_df = df.iloc[:, 0:54]
clustering_cff_df = summary_df.iloc[:, 0::6]
clustering_cff_mean, clustering_cff_sem = clustering_cff_df.mean(), clustering_cff_df.sem()
axes[0][0].errorbar(x, clustering_cff_mean, yerr=clustering_cff_sem, capsize=2, color='0.4')
axes[0][0].plot(x, clustering_cff_mean, marker="s", markersize=6, color='0.4')
axes[0][0].set_title('clustering coefficient')
axes[0][0].set_xticklabels(x, rotation=70)
axes[0][0].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

shortest_path_df = summary_df.iloc[:, 1::6]
shortest_path_mean, shortest_path_sem = shortest_path_df.mean(), shortest_path_df.sem()
axes[0][1].errorbar(x, shortest_path_mean, yerr=shortest_path_sem, capsize=2, color='0.4')
axes[0][1].plot(x, shortest_path_mean, marker="s", markersize=6, color='0.4')
axes[0][1].set_title('shortest path length')
axes[0][1].set_xticklabels(x, rotation=70)
axes[0][1].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

cofiring_strength_df = summary_df.iloc[:, 2::6]
cofiring_strength_mean, cofiring_strength_sem = cofiring_strength_df.mean(), cofiring_strength_df.sem()
axes[0][2].errorbar(x, cofiring_strength_mean, yerr=cofiring_strength_sem, capsize=2, color='0.4')
axes[0][2].plot(x, cofiring_strength_mean, marker="s", markersize=6, color='0.4')
axes[0][2].set_title('cofiring strength')
axes[0][2].set_xticklabels(x, rotation=70)
axes[0][2].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

total_weight_df = summary_df.iloc[:, 3::6]
total_weight_mean, total_weight_sem = total_weight_df.mean(), total_weight_df.sem()
axes[1][0].errorbar(x, total_weight_mean, yerr=total_weight_sem, capsize=2, color='0.4')
axes[1][0].plot(x, total_weight_mean, marker="s", markersize=6, color='0.4')
axes[1][0].set_title('average weight')
axes[1][0].set_xticklabels(x, rotation=70)
axes[1][0].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

total_edge_df = summary_df.iloc[:, 4::6]
total_edge_mean, total_edge_sem = total_edge_df.mean(), total_edge_df.sem()
axes[1][1].errorbar(x, total_edge_mean, yerr=total_edge_sem, capsize=2, color='0.4')
axes[1][1].plot(x, total_edge_mean, marker="s", markersize=6, color='0.4')
axes[1][1].set_title('average edges')
axes[1][1].set_xticklabels(x, rotation=70)
axes[1][1].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

assortativity_df = summary_df.iloc[:, 5::6]
assortativity_mean, assortativity_sem = assortativity_df.mean(), assortativity_df.sem()
axes[1][2].errorbar(x, assortativity_mean, yerr=assortativity_sem, capsize=2, color='0.4')
axes[1][2].plot(x, assortativity_mean, marker="s", markersize=6, color='0.4')
axes[1][2].set_title('assortativity')
axes[1][2].set_xticklabels(x, rotation=70)
axes[1][2].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

fig.show()

In [ ]:
df_without_extinction = df.drop(columns=df.columns[list(range(12, 36))])
df_without_extinction

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(8, 6), tight_layout=True, dpi=200)

x = ['A1pre', 'A1post', 'A4pre', 'A4post', 'A5']
a = [0, 1, 2, 3, 4, 6, 7, 8, 9]
summary_df = df_without_extinction.iloc[:, 0:30]
clustering_cff_df = summary_df.iloc[:, 0::6]
clustering_cff_mean, clustering_cff_sem = clustering_cff_df.mean(), clustering_cff_df.sem()
axes[0][0].errorbar(x, clustering_cff_mean, yerr=clustering_cff_sem, capsize=2, color='0.4')
axes[0][0].plot(x, clustering_cff_mean, marker="s", markersize=6, color='0.4')
axes[0][0].set_title('clustering coefficient')
axes[0][0].set_xticklabels(x, rotation=70)
axes[0][0].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

shortest_path_df = summary_df.iloc[:, 1::6]
shortest_path_mean, shortest_path_sem = shortest_path_df.mean(), shortest_path_df.sem()
axes[0][1].errorbar(x, shortest_path_mean, yerr=shortest_path_sem, capsize=2, color='0.4')
axes[0][1].plot(x, shortest_path_mean, marker="s", markersize=6, color='0.4')
axes[0][1].set_title('shortest path length')
axes[0][1].set_xticklabels(x, rotation=70)
axes[0][1].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

cofiring_strength_df = summary_df.iloc[:, 2::6]
cofiring_strength_mean, cofiring_strength_sem = cofiring_strength_df.mean(), cofiring_strength_df.sem()
axes[0][2].errorbar(x, cofiring_strength_mean, yerr=cofiring_strength_sem, capsize=2, color='0.4')
axes[0][2].plot(x, cofiring_strength_mean, marker="s", markersize=6, color='0.4')
axes[0][2].set_title('cofiring strength')
axes[0][2].set_xticklabels(x, rotation=70)
axes[0][2].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

total_weight_df = summary_df.iloc[:, 3::6]
total_weight_mean, total_weight_sem = total_weight_df.mean(), total_weight_df.sem()
axes[1][0].errorbar(x, total_weight_mean, yerr=total_weight_sem, capsize=2, color='0.4')
axes[1][0].plot(x, total_weight_mean, marker="s", markersize=6, color='0.4')
axes[1][0].set_title('average weight')
axes[1][0].set_xticklabels(x, rotation=70)
axes[1][0].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

total_edge_df = summary_df.iloc[:, 4::6]
total_edge_mean, total_edge_sem = total_edge_df.mean(), total_edge_df.sem()
axes[1][1].errorbar(x, total_edge_mean, yerr=total_edge_sem, capsize=2, color='0.4')
axes[1][1].plot(x, total_edge_mean, marker="s", markersize=6, color='0.4')
axes[1][1].set_title('average edges')
axes[1][1].set_xticklabels(x, rotation=70)
axes[1][1].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

assortativity_df = summary_df.iloc[:, 5::6]
assortativity_mean, assortativity_sem = assortativity_df.mean(), assortativity_df.sem()
axes[1][2].errorbar(x, assortativity_mean, yerr=assortativity_sem, capsize=2, color='0.4')
axes[1][2].plot(x, assortativity_mean, marker="s", markersize=6, color='0.4')
axes[1][2].set_title('assortativity')
axes[1][2].set_xticklabels(x, rotation=70)
axes[1][2].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

fig.show()

In [ ]:
sorted_df = df_without_extinction[['A1pre', 'A1_1', 'A1_2', 'A1_3', 'A4pre', 'A4_1', 'A4_2', 'A4_3', 'A5']]
sorted_df

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(8, 6), tight_layout=True, dpi=200)

x = ['A1pre', 'A1_1', 'A1_2', 'A1_3', 'A4pre', 'A4_1', 'A4_2', 'A4_3', 'A5']
a = [0, 1, 2, 3, 4, 6, 7, 8, 9]
summary_df = sorted_df.iloc[:, 0:54]
clustering_cff_df = summary_df.iloc[:, 0::6]
clustering_cff_mean, clustering_cff_sem = clustering_cff_df.mean(), clustering_cff_df.sem()
axes[0][0].errorbar(x, clustering_cff_mean, yerr=clustering_cff_sem, capsize=2, color='0.4')
axes[0][0].plot(x, clustering_cff_mean, marker="s", markersize=6, color='0.4')
axes[0][0].set_title('clustering coefficient')
axes[0][0].set_xticklabels(x, rotation=70)
axes[0][0].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

shortest_path_df = summary_df.iloc[:, 1::6]
shortest_path_mean, shortest_path_sem = shortest_path_df.mean(), shortest_path_df.sem()
axes[0][1].errorbar(x, shortest_path_mean, yerr=shortest_path_sem, capsize=2, color='0.4')
axes[0][1].plot(x, shortest_path_mean, marker="s", markersize=6, color='0.4')
axes[0][1].set_title('shortest path length')
axes[0][1].set_xticklabels(x, rotation=70)
axes[0][1].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

cofiring_strength_df = summary_df.iloc[:, 2::6]
cofiring_strength_mean, cofiring_strength_sem = cofiring_strength_df.mean(), cofiring_strength_df.sem()
axes[0][2].errorbar(x, cofiring_strength_mean, yerr=cofiring_strength_sem, capsize=2, color='0.4')
axes[0][2].plot(x, cofiring_strength_mean, marker="s", markersize=6, color='0.4')
axes[0][2].set_title('cofiring strength')
axes[0][2].set_xticklabels(x, rotation=70)
axes[0][2].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

total_weight_df = summary_df.iloc[:, 3::6]
total_weight_mean, total_weight_sem = total_weight_df.mean(), total_weight_df.sem()
axes[1][0].errorbar(x, total_weight_mean, yerr=total_weight_sem, capsize=2, color='0.4')
axes[1][0].plot(x, total_weight_mean, marker="s", markersize=6, color='0.4')
axes[1][0].set_title('average weight')
axes[1][0].set_xticklabels(x, rotation=70)
axes[1][0].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

total_edge_df = summary_df.iloc[:, 4::6]
total_edge_mean, total_edge_sem = total_edge_df.mean(), total_edge_df.sem()
axes[1][1].errorbar(x, total_edge_mean, yerr=total_edge_sem, capsize=2, color='0.4')
axes[1][1].plot(x, total_edge_mean, marker="s", markersize=6, color='0.4')
axes[1][1].set_title('average edges')
axes[1][1].set_xticklabels(x, rotation=70)
axes[1][1].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

assortativity_df = summary_df.iloc[:, 5::6]
assortativity_mean, assortativity_sem = assortativity_df.mean(), assortativity_df.sem()
axes[1][2].errorbar(x, assortativity_mean, yerr=assortativity_sem, capsize=2, color='0.4')
axes[1][2].plot(x, assortativity_mean, marker="s", markersize=6, color='0.4')
axes[1][2].set_title('assortativity')
axes[1][2].set_xticklabels(x, rotation=70)
axes[1][2].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

fig.show()

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=3, figsize=(8, 8), tight_layout=True, dpi=200)

# A1
x = ['A1pre', 'A1post']
a = [0, 1, 2, 3, 4, 6, 7, 8, 9]
summary_df = df.iloc[a, 0:12]
clustering_cff_df = summary_df.iloc[:, 0::6]
clustering_cff_mean, clustering_cff_sem = clustering_cff_df.mean(), clustering_cff_df.sem()
axes[0][0].errorbar(x, clustering_cff_mean, yerr=clustering_cff_sem, capsize=2, color='0.4')
axes[0][0].plot(x, clustering_cff_mean, marker="s", markersize=6, color='0.4')
axes[0][0].set_title('clustering coefficient')
axes[0][0].set_xticklabels(x, rotation=45)

shortest_path_df = summary_df.iloc[:, 1::6]
shortest_path_mean, shortest_path_sem = shortest_path_df.mean(), shortest_path_df.sem()
axes[0][1].errorbar(x, shortest_path_mean, yerr=shortest_path_sem, capsize=2, color='0.4')
axes[0][1].plot(x, shortest_path_mean, marker="s", markersize=6, color='0.4')
axes[0][1].set_title('shortest path length')
axes[0][1].set_xticklabels(x, rotation=45)

cofiring_strength_df = summary_df.iloc[:, 2::6]
cofiring_strength_mean, cofiring_strength_sem = cofiring_strength_df.mean(), cofiring_strength_df.sem()
axes[0][2].errorbar(x, cofiring_strength_mean, yerr=cofiring_strength_sem, capsize=2, color='0.4')
axes[0][2].plot(x, cofiring_strength_mean, marker="s", markersize=6, color='0.4')
axes[0][2].set_title('cofiring strength')
axes[0][2].set_xticklabels(x, rotation=45)

# A2, A3

x = ['A2_start', 'A2_end', 'A3_start', 'A3_end']
a = [0, 1, 2, 3, 4, 6, 7, 8, 9]
summary_df = df.iloc[a, 12:36]
clustering_cff_df = summary_df.iloc[:, 0::6]
clustering_cff_mean, clustering_cff_sem = clustering_cff_df.mean(), clustering_cff_df.sem()
axes[1][0].errorbar(x, clustering_cff_mean, yerr=clustering_cff_sem, capsize=2, color='0.4')
axes[1][0].plot(x, clustering_cff_mean, marker="s", markersize=6, color='0.4')
axes[1][0].set_title('clustering coefficient')
axes[1][0].set_xticklabels(x, rotation=45)

shortest_path_df = summary_df.iloc[:, 1::6]
shortest_path_mean, shortest_path_sem = shortest_path_df.mean(), shortest_path_df.sem()
axes[1][1].errorbar(x, shortest_path_mean, yerr=shortest_path_sem, capsize=2, color='0.4')
axes[1][1].plot(x, shortest_path_mean, marker="s", markersize=6, color='0.4')
axes[1][1].set_title('shortest path length')
axes[1][1].set_xticklabels(x, rotation=45)

cofiring_strength_df = summary_df.iloc[:, 2::6]
cofiring_strength_mean, cofiring_strength_sem = cofiring_strength_df.mean(), cofiring_strength_df.sem()
axes[1][2].errorbar(x, cofiring_strength_mean, yerr=cofiring_strength_sem, capsize=2, color='0.4')
axes[1][2].plot(x, cofiring_strength_mean, marker="s", markersize=6, color='0.4')
axes[1][2].set_title('cofiring strength')
axes[1][2].set_xticklabels(x, rotation=45)

# A4, A5

x = ['A4pre', 'A4post', 'A5']
a = [0, 1, 2, 3, 4, 6, 7, 8, 9]
summary_df = df.iloc[a, 36:54]
clustering_cff_df = summary_df.iloc[:, 0::6]
clustering_cff_mean, clustering_cff_sem = clustering_cff_df.mean(), clustering_cff_df.sem()
axes[2][0].errorbar(x, clustering_cff_mean, yerr=clustering_cff_sem, capsize=2, color='0.4')
axes[2][0].plot(x, clustering_cff_mean, marker="s", markersize=6, color='0.4')
axes[2][0].set_title('clustering coefficient')
axes[2][0].set_xticklabels(x, rotation=45)

shortest_path_df = summary_df.iloc[:, 1::6]
shortest_path_mean, shortest_path_sem = shortest_path_df.mean(), shortest_path_df.sem()
axes[2][1].errorbar(x, shortest_path_mean, yerr=shortest_path_sem, capsize=2, color='0.4')
axes[2][1].plot(x, shortest_path_mean, marker="s", markersize=6, color='0.4')
axes[2][1].set_title('shortest path length')
axes[2][1].set_xticklabels(x, rotation=45)

cofiring_strength_df = summary_df.iloc[:, 2::6]
cofiring_strength_mean, cofiring_strength_sem = cofiring_strength_df.mean(), cofiring_strength_df.sem()
axes[2][2].errorbar(x, cofiring_strength_mean, yerr=cofiring_strength_sem, capsize=2, color='0.4')
axes[2][2].plot(x, cofiring_strength_mean, marker="s", markersize=6, color='0.4')
axes[2][2].set_title('cofiring strength')
axes[2][2].set_xticklabels(x, rotation=45)

fig.show()

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(8, 6), tight_layout=True, dpi=200)

x = ['A1_1', 'A1_2', 'A1_3', 'A4_1', 'A4_2', 'A4_3']
a = [0, 1, 2, 3, 4, 6, 7, 8, 9]
summary_df = df.iloc[:, 54:]
clustering_cff_df = summary_df.iloc[:, 0::6]
clustering_cff_mean, clustering_cff_sem = clustering_cff_df.mean(), clustering_cff_df.sem()
axes[0][0].errorbar(x, clustering_cff_mean, yerr=clustering_cff_sem, capsize=2, color='0.4')
axes[0][0].plot(x, clustering_cff_mean, marker="s", markersize=6, color='0.4')
axes[0][0].set_title('clustering coefficient')
axes[0][0].set_xticklabels(x, rotation=70)
axes[0][0].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

shortest_path_df = summary_df.iloc[:, 1::6]
shortest_path_mean, shortest_path_sem = shortest_path_df.mean(), shortest_path_df.sem()
axes[0][1].errorbar(x, shortest_path_mean, yerr=shortest_path_sem, capsize=2, color='0.4')
axes[0][1].plot(x, shortest_path_mean, marker="s", markersize=6, color='0.4')
axes[0][1].set_title('shortest path length')
axes[0][1].set_xticklabels(x, rotation=70)
axes[0][1].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

cofiring_strength_df = summary_df.iloc[:, 2::6]
cofiring_strength_mean, cofiring_strength_sem = cofiring_strength_df.mean(), cofiring_strength_df.sem()
axes[0][2].errorbar(x, cofiring_strength_mean, yerr=cofiring_strength_sem, capsize=2, color='0.4')
axes[0][2].plot(x, cofiring_strength_mean, marker="s", markersize=6, color='0.4')
axes[0][2].set_title('cofiring strength')
axes[0][2].set_xticklabels(x, rotation=70)
axes[0][2].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

total_weight_df = summary_df.iloc[:, 3::6]
total_weight_mean, total_weight_sem = total_weight_df.mean(), total_weight_df.sem()
axes[1][0].errorbar(x, total_weight_mean, yerr=total_weight_sem, capsize=2, color='0.4')
axes[1][0].plot(x, total_weight_mean, marker="s", markersize=6, color='0.4')
axes[1][0].set_title('average weight')
axes[1][0].set_xticklabels(x, rotation=70)
axes[1][0].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

total_edge_df = summary_df.iloc[:, 4::6]
total_edge_mean, total_edge_sem = total_edge_df.mean(), total_edge_df.sem()
axes[1][1].errorbar(x, total_edge_mean, yerr=total_edge_sem, capsize=2, color='0.4')
axes[1][1].plot(x, total_edge_mean, marker="s", markersize=6, color='0.4')
axes[1][1].set_title('average edges')
axes[1][1].set_xticklabels(x, rotation=70)
axes[1][1].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

assortativity_df = summary_df.iloc[:, 5::6]
assortativity_mean, assortativity_sem = assortativity_df.mean(), assortativity_df.sem()
axes[1][2].errorbar(x, assortativity_mean, yerr=assortativity_sem, capsize=2, color='0.4')
axes[1][2].plot(x, assortativity_mean, marker="s", markersize=6, color='0.4')
axes[1][2].set_title('assortativity')
axes[1][2].set_xticklabels(x, rotation=70)
axes[1][2].grid(which = "major", axis = "x", color = "0.7", alpha = 0.6, linestyle = "--", linewidth = 1)

fig.show()

In [ ]:
df.iloc[:, 54:].describe().iloc[:, 2::6]

In [ ]:
from scipy.stats import f_oneway

# clustering coefficient
a1_pre = df.iloc[:, 0]
a1_post = df.iloc[:, 6]
# a2_start = df.iloc[:, 12]
# a2_end = df.iloc[:, 18]
# a3_start = df.iloc[:, 24]
# a3_end = df.iloc[:, 30]
a4_pre = df.iloc[:, 36]
a4_post = df.iloc[:, 42]
a5 = df.iloc[:, 48]

_, pvalue = f_oneway(a1_pre, a1_post, a4_pre, a4_post, a5)
print('pvalue of clustering coefficient: %f' % pvalue)

# shortest path length
a1_pre = df.iloc[:, 1]
a1_post = df.iloc[:, 7]
# a2_start = df.iloc[:, 13]
# a2_end = df.iloc[:, 19]
# a3_start = df.iloc[:, 25]
# a3_end = df.iloc[:, 31]
a4_pre = df.iloc[:, 37]
a4_post = df.iloc[:, 43]
a5 = df.iloc[:, 49]

_, pvalue = f_oneway(a1_pre, a1_post, a4_pre, a4_post, a5)
print('pvalue of shortest path length: %f' % pvalue)

# cofiring strength
a1_pre = df.iloc[:, 2]
a1_post = df.iloc[:, 8]
# a2_start = df.iloc[:, 9]
# a2_end = df.iloc[:, 20]
# a3_start = df.iloc[:, 26]
# a3_end = df.iloc[:, 32]
a4_pre = df.iloc[:, 38]
a4_post = df.iloc[:, 44]
a5 = df.iloc[:, 50]

_, pvalue = f_oneway(a1_pre, a1_post, a4_pre, a4_post, a5)
print('pvalue of cofiring strength: %f' % pvalue)

# assortativity
a1_pre = df.iloc[:, 5].fillna(0.0)
a1_post = df.iloc[:, 11].fillna(0.0)
# a2_start = df.iloc[:, 17].fillna(0.0)
# a2_end = df.iloc[:, 23].fillna(0.0)
# a3_start = df.iloc[:, 29].fillna(0.0)
# a3_end = df.iloc[:, 35].fillna(0.0)
a4_pre = df.iloc[:, 41].fillna(0.0)
a4_post = df.iloc[:, 47].fillna(0.0)
a5 = df.iloc[:, 53].fillna(0.0)

_, pvalue = f_oneway(a1_pre, a1_post, a4_pre, a4_post, a5)
print('pvalue of assortativity: %f' % pvalue)

In [ ]:
df.iloc[:, 54:]